In [ ]:
from __future__ import division
from numpy import *
from pylab import *
from numpy.random import rand
import scipy.stats
import  scipy.linalg
from scipy.optimize.lbfgsb import fmin_l_bfgs_b


##########################################################################################################################################################
def historic_vol(currency,tenor):

    #Volatilidad historica estandar
            
    M = currency.shape[0]   #No de dias que tenemos del spot
    
    #Definimos un vector de retornos logaritmicos
    log_return = zeros((M-1,1))

    #Definimos el numero de observaciones moviles a usar dependiendo del plazo
    if tenor == '1W':
        obs = 5
    if tenor == '2W':
        obs = 10
    elif tenor == '1M':
        obs = 21
    elif tenor == '2M':
        obs = 43
    elif tenor == '3M':
        obs = 65
    elif tenor == '6M':
        obs = 129
    elif tenor == '9M':
        obs = 194
    elif tenor == '1Y':
        obs = 260
    elif tenor == '2Y':
        obs = 520

    N = M-obs     #No de retornos que usaremos para calcular la volatilidad. Perdemos el numero obs  
  
    #Calculamos los retornos
    for i in xrange(0,M-1):
        log_return[i] = log(currency[i+1,4])-log(currency[i,4])

    #Definimos una matriz para almacenar los calculos
    calculations = zeros((N,4))

    for i in xrange(0,N):
        calculations[i,0] = log_return[i+obs-1]
        calculations[i,1] = mean(log_return[i:i+obs])
        calculations[i,2] = (calculations[i,0] - calculations[i,1])**2

    for i in xrange(obs-1,N):
        calculations[i,3] = (252**0.5)*(sum(calculations[i-obs+1:i+1,2])/(size(calculations[i-obs+1:i+1,2])-1))**0.5

    #Definicion de un vector con las fechas
    fechas = empty((N,1),dtype =datetime.date)
        
    for i in xrange(obs,M):
        fechas[i-obs] = datetime.date(int(currency[i,0][6:10]), int(currency[i,0][3:5]), int(currency[i,0][0:2]))

    #Definimos una matrix de resultados finales
    vols_standar = zeros((N,2))

    #vols_standar[:,0] = currency[obs:M,0]
    vols_standar[1:N,0] = log_return[obs:M,0]
    vols_standar[:,1] = calculations[:,3]

    #Creacion de la matriz completa
    
    vols_standar=concatenate((fechas,vols_standar),axis=1)
    
    return  vols_standar    
